In [2]:
import os, json
import pandas as pd

import utils as u
import etl

cfg = u.get_config()
data_cfg = u.get_config("data")
csv_json_data_dir = f"{cfg['DIRS']['CSV_OF_JSON_DATA']}"

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

In [2]:
match_stats_raw = pd.read_csv(os.path.join(csv_json_data_dir, "raw/match_stats.csv"))
match_stats_raw = match_stats_raw[(match_stats_raw.league_type == "League") | (match_stats_raw.country == "World")]
print("Total domestic league and UEFA matches:", match_stats_raw.shape)
match_stats_raw = match_stats_raw[match_stats_raw.league_season >= 2016]
print("Total matches in seasons of interest:", match_stats_raw.shape)
match_stats_raw = match_stats_raw[
    (match_stats_raw.league_type == "League") |
    ((match_stats_raw.country == "World") & (match_stats_raw.fixture_round.isin(data_cfg["WORLD_DATA"]["KEY_FIXTURE_ROUNDS"])))]
print("Total matches in rounds of interest:", match_stats_raw.shape)
match_stats_raw = match_stats_raw[match_stats_raw.fixture_status == "Match Finished"]
print("Total matches that have actually been played:", match_stats_raw.shape)
match_stats_raw = match_stats_raw[~match_stats_raw[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isna().all(axis=1)]
print("Total matches with at least some stats:", match_stats_raw.shape)
at_least_one_missing = match_stats_raw[match_stats_raw[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isna().any(axis=1)]
print("Proportion of matches with at least one missing data point: ", round(len(at_least_one_missing)/len(match_stats_raw), 3)*100, "%", sep="")
print("Number of missing data points:", at_least_one_missing[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isnull().sum().sum())
display(pd.DataFrame(match_stats_raw.league_name.value_counts()))

Total domestic league and UEFA matches: (20528, 55)
Total matches in seasons of interest: (9582, 55)
Total matches in rounds of interest: (8586, 55)
Total matches that have actually been played: (7999, 55)
Total matches with at least some stats: (7502, 55)
Proportion of matches with at least one missing data point: 32.4%
Number of missing data points: 7301


,league_name
Premier League,1422
Ligue 1,1391
Serie A,1369
Primera Division,1356
Bundesliga 1,1129
Champions League,454
Europa League,381


In [3]:
match_stats_raw1 = match_stats_raw.copy()
# Operations to fill in missing data using other columns
# 1) Fix for one match
match_stats_raw1.loc[((match_stats_raw1.fixture_id).astype(int) == 21650), 'home_goals'] = 2
match_stats_raw1.loc[((match_stats_raw1.fixture_id).astype(int) == 21650), 'away_goals'] = 1
match_stats_raw1.loc[((match_stats_raw1.fixture_id).astype(int) == 21650), 'fixture_result_ht'] = '1-0'
match_stats_raw1.loc[((match_stats_raw1.fixture_id).astype(int) == 21650), 'fixture_result_ft'] = '2-1'
# 2) Fix shots_ont
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_ont", "away_shots_tot"]].isnull().sum().values)
match_stats_raw1['home_shots_tot'].fillna(match_stats_raw1['home_shots_ont'] + match_stats_raw1['home_shots_offt'] + match_stats_raw1['home_shots_bl'], inplace=True)
match_stats_raw1['away_shots_tot'].fillna(match_stats_raw1['away_shots_ont'] + match_stats_raw1['away_shots_offt'] + match_stats_raw1['away_shots_bl'], inplace=True)
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_ont", "away_shots_tot"]].isnull().sum().values)
# 3) Fix shots_bl
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_bl", "away_shots_bl"]].isnull().sum().values)
match_stats_raw1['home_shots_bl'].fillna(match_stats_raw1['home_shots_tot'] - (match_stats_raw1['home_shots_ont'] + match_stats_raw1['home_shots_offt']), inplace=True)
match_stats_raw1['away_shots_bl'].fillna(match_stats_raw1['away_shots_tot'] - (match_stats_raw1['away_shots_ont'] + match_stats_raw1['away_shots_offt']), inplace=True)
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_bl", "away_shots_bl"]].isnull().sum().values)
# 4) Fix shots_outb
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_outb", "away_shots_outb"]].isnull().sum().values)
match_stats_raw1.loc[match_stats_raw1["fixture_id"]==20592,'home_shots_outb'] = (
    match_stats_raw1.loc[match_stats_raw1["fixture_id"]==20592,'home_shots_tot'] - match_stats_raw1.loc[match_stats_raw1["fixture_id"]==20592,'home_shots_inb'])
match_stats_raw1.loc[match_stats_raw1["fixture_id"]==20592,'away_shots_outb'] = (
    match_stats_raw1.loc[match_stats_raw1["fixture_id"]==20592,'away_shots_tot'] - match_stats_raw1.loc[match_stats_raw1["fixture_id"]==20592,'away_shots_inb'])
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_outb", "away_shots_outb"]].isnull().sum().values)
# 5) Fix shots_inb
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_inb", "away_shots_inb"]].isnull().sum().values)
match_stats_raw1['home_shots_inb'] = (match_stats_raw1['home_shots_tot'] - match_stats_raw1['home_shots_outb'])
match_stats_raw1['away_shots_inb'] = (match_stats_raw1['away_shots_tot'] - match_stats_raw1['away_shots_outb'])
print(match_stats_raw1.shape, match_stats_raw1[["home_shots_inb", "away_shots_inb"]].isnull().sum().values)
# 6) Fix passes_pct
print(match_stats_raw1.shape, match_stats_raw1[["home_passes_pct", "away_passes_pct"]].isnull().sum().values)
match_stats_raw1['home_passes_pct'] = (match_stats_raw1['home_passes_acc']/match_stats_raw1['home_passes_tot']).apply(lambda x: round(x, 2))
match_stats_raw1['away_passes_pct'] = (match_stats_raw1['away_passes_acc']/match_stats_raw1['away_passes_tot']).apply(lambda x: round(x, 2))
print(match_stats_raw1.shape, match_stats_raw1[["home_passes_pct", "away_passes_pct"]].isnull().sum().values)

(7502, 55) [0 2]
(7502, 55) [0 0]
(7502, 55) [24 24]
(7502, 55) [0 0]
(7502, 55) [3 3]
(7502, 55) [2 2]
(7502, 55) [2 2]
(7502, 55) [2 2]
(7502, 55) [1333 1333]
(7502, 55) [2 2]


In [4]:
match_stats_raw2 = match_stats_raw1.copy()
# Operations to fill in missing data using match events data
# 1) Setup of counts form match_events
match_events_count_clean = pd.read_csv(os.path.join(csv_json_data_dir, "clean/match_events_count.csv"))
match_events_count_clean = match_events_count_clean.assign(
    home_goals = (match_events_count_clean['home_normal_goal'] + match_events_count_clean['home_own_goal'] + match_events_count_clean['home_penalty']),
    away_goals = (match_events_count_clean['away_normal_goal'] + match_events_count_clean['away_own_goal'] + match_events_count_clean['away_penalty']))
key_match_counts = match_events_count_clean[['fixture_id', 'home_goals', 'away_goals', 'home_red_card', 'home_yellow_card','away_red_card', 'away_yellow_card']]
relevant_fixture_ids = match_stats_raw2.fixture_id.unique().tolist()
key_match_counts = key_match_counts.loc[key_match_counts.fixture_id.isin(relevant_fixture_ids)].copy().reset_index(drop=True)
print(key_match_counts.shape)
key_match_counts.sort_values(by='fixture_id', inplace=True)
print(key_match_counts.shape)
print(match_stats_raw2.shape)
match_stats_raw2.reset_index(drop=True, inplace=True)
match_stats_raw2.sort_values(by='fixture_id', inplace=True)
print(match_stats_raw2.shape)
# 2) Fill in data
print(match_stats_raw2.shape, match_stats_raw2[["home_goals", "away_goals", "home_yc", "away_yc", "home_rc", "away_rc"]].isnull().sum().values)
match_stats_raw2 = match_stats_raw2.assign(
    home_goals=key_match_counts['home_goals'], home_rc=key_match_counts['home_red_card'], home_yc=key_match_counts['home_yellow_card'],
    away_goals=key_match_counts['away_goals'], away_rc=key_match_counts['away_red_card'], away_yc=key_match_counts['away_yellow_card'])
print(match_stats_raw2.shape, match_stats_raw2[["home_goals", "away_goals", "home_yc", "away_yc", "home_rc", "away_rc"]].isnull().sum().values)
match_stats_raw2_nulls = pd.DataFrame(match_stats_raw2[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isnull().sum())
at_least_one_missing2 = match_stats_raw2[match_stats_raw2[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isna().any(axis=1)]
print("Proportion of matches with at least one missing data point: ", round(len(at_least_one_missing2)/len(match_stats_raw), 2)*100, "%", sep="")
print("Number of missing data points:", at_least_one_missing2[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isnull().sum().sum())
display("Showing missing data by columns", pd.DataFrame(match_stats_raw2_nulls[match_stats_raw2_nulls[0]>0]))
display("Missing offsides data by competition:\n", pd.DataFrame(match_stats_raw2[match_stats_raw2.home_offsides.isnull()].league_name.value_counts()))
display("Missing gksaves data by competition:\n", pd.DataFrame(match_stats_raw2[match_stats_raw2.home_gksaves.isnull()].league_name.value_counts()))
display("Missing shots/passed data by competition:\n", pd.DataFrame(match_stats_raw2[match_stats_raw2.home_passes_tot.isnull()].league_name.value_counts()))

(7502, 7)
(7502, 7)
(7502, 55)
(7502, 55)
(7502, 55) [   0    0  233  233 1790 1777]
(7502, 55) [0 0 0 0 0 0]
Proportion of matches with at least one missing data point: 4.0%
Number of missing data points: 552


'Showing missing data by columns'

,0
home_shots_inb,2
away_shots_inb,2
home_shots_outb,2
away_shots_outb,2
home_passes_acc,2
away_passes_acc,2
home_passes_tot,2
away_passes_tot,2
home_passes_pct,2
away_passes_pct,2


'Missing offsides data by competition:\n'

,league_name
Ligue 1,67
Premier League,53
Serie A,41
Bundesliga 1,37
Primera Division,26
Champions League,10
Europa League,6


'Missing gksaves data by competition:\n'

,league_name
Premier League,6
Serie A,5
Primera Division,5
Ligue 1,5
Europa League,2
Bundesliga 1,2
Champions League,1


'Missing shots/passed data by competition:\n'

,league_name
Serie A,1
Primera Division,1


In [5]:
match_stats_raw3 = match_stats_raw2.copy()
# Operations to only keep matches of teams of interest
key_team_ids = u.get_ids_of_key_teams()
print("Total matches before filtering by what teams are playing:", match_stats_raw3.shape)
display(pd.DataFrame(match_stats_raw3[match_stats_raw3.country == "World"].league_name.value_counts()))
# 1) Matches with at least one team of interest playing
match_stats_raw3 = match_stats_raw3[(match_stats_raw3['home_team_id'].isin(key_team_ids)) | (match_stats_raw3['away_team_id'].isin(key_team_ids))].copy()
print("Total matches where at least one team is from one of the 5 major European Leagues:", match_stats_raw3.shape)
display(pd.DataFrame(match_stats_raw3[match_stats_raw3.country == "World"].league_name.value_counts()))
# 2) Matches where both teams playing are of interest
match_stats_raw3 = match_stats_raw3[(match_stats_raw3['home_team_id'].isin(key_team_ids)) & (match_stats_raw3['away_team_id'].isin(key_team_ids))].copy()
print("Total matches where BOTH teams are from one of the 5 major European Leagues:", match_stats_raw3.shape)
display(pd.DataFrame(match_stats_raw3[match_stats_raw3.country == "World"].league_name.value_counts()))

Total matches before filtering by what teams are playing: (7502, 55)


,league_name
Champions League,454
Europa League,381


Total matches where at least one team is from one of the 5 major European Leagues: (7270, 55)


,league_name
Champions League,396
Europa League,207


Total matches where BOTH teams are from one of the 5 major European Leagues: (6864, 55)


,league_name
Champions League,162
Europa League,35


In [6]:
# Investigating how many of the UEFA competition matches we have and how much we lose due to data issues
match_stats_uefa = pd.read_csv(os.path.join(csv_json_data_dir, "raw/match_stats.csv"))
match_stats_uefa = match_stats_uefa[(match_stats_uefa.country == "World")]
print("\nTotal number of UEFA matches:", match_stats_uefa.shape, "Breakdown:")
uefa_grouped1 = match_stats_uefa.groupby(by=["league_name", "league_season"]).count()[["fixture_id"]].rename(columns={"fixture_id":"ngames"}).reset_index().sort_values(by=["league_name", "league_season"], ascending=False)
display(pd.DataFrame(uefa_grouped1))

match_stats_uefa = match_stats_uefa[match_stats_uefa.league_season >= 2016].copy()
print("\nTotal UEFA matches in seasons of interest:", match_stats_uefa.shape, "Breakdown:")
uefa_grouped2 = match_stats_uefa.groupby(by=["league_name", "league_season"]).count()[["fixture_id"]].rename(columns={"fixture_id":"ngames"}).reset_index().sort_values(by=["league_name", "league_season"], ascending=False)
uefa_grouped2["matches_lost"] = uefa_grouped1["ngames"].values - uefa_grouped2["ngames"].values
display(uefa_grouped2)

match_stats_uefa = match_stats_uefa[match_stats_uefa.fixture_round.isin(data_cfg["WORLD_DATA"]["KEY_FIXTURE_ROUNDS"])]
print("\nTotal UEFA matches in rounds of interest:", match_stats_uefa.shape, "Breakdown:")
uefa_grouped3 = match_stats_uefa.groupby(by=["league_name", "league_season"]).count()[["fixture_id"]].rename(columns={"fixture_id":"ngames"}).reset_index().sort_values(by=["league_name", "league_season"], ascending=False)
uefa_grouped3["matches_lost"] = uefa_grouped2["ngames"].values - uefa_grouped3["ngames"].values
display(uefa_grouped3)

match_stats_uefa = match_stats_uefa[match_stats_uefa.fixture_status == "Match Finished"]
print("\nTotal UEFA matches that have actually been played:", match_stats_uefa.shape, "Breakdown:")
uefa_grouped4 = match_stats_uefa.groupby(by=["league_name", "league_season"]).count()[["fixture_id"]].rename(columns={"fixture_id":"ngames"}).reset_index().sort_values(by=["league_name", "league_season"], ascending=False)
uefa_grouped4["matches_lost"] = uefa_grouped3["ngames"].values - uefa_grouped4["ngames"].values
display(uefa_grouped4)

uefa_dropped = match_stats_uefa[match_stats_uefa[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isna().all(axis=1)]
match_stats_uefa = match_stats_uefa[~match_stats_uefa[data_cfg['MATCH_STAT']['COLUMNS'][1:]].isna().all(axis=1)]
print("\nTotal UEFA matches with at least some stats:", match_stats_uefa.shape, "Breakdown:")
print("Total UEFA matches with stats in the corrected stats df:", match_stats_raw2[match_stats_raw2.country == "World"].shape)
match_stats_uefa = match_stats_raw2[match_stats_raw2.country == "World"].copy()
uefa_grouped5 = match_stats_uefa.groupby(by=["league_name", "league_season"]).count()[["fixture_id"]].rename(columns={"fixture_id":"ngames"}).reset_index().sort_values(by=["league_name", "league_season"], ascending=False)
uefa_grouped5["matches_lost"] = uefa_grouped4.loc[[6,7,0,1,2,3],].sort_values(by=["league_name", "league_season"], ascending=False)["ngames"].values - uefa_grouped5["ngames"].values
display(uefa_grouped5)
print("Showing how many null values are in our final dataset")
uefa_null = pd.DataFrame(match_stats_raw2[match_stats_raw2.country == "World"][data_cfg['MATCH_STAT']['COLUMNS'][1:]].isnull().sum())
display(uefa_null[uefa_null[0]>0])


Total number of UEFA matches: (2274, 55) Breakdown:


,league_name,league_season,ngames
7,Europa League,2019,490
6,Europa League,2018,519
5,Europa League,2017,205
4,Europa League,2016,205
3,Champions League,2019,203
2,Champions League,2018,216
1,Champions League,2017,219
0,Champions League,2016,217



Total UEFA matches in seasons of interest: (2274, 55) Breakdown:


,league_name,league_season,ngames,matches_lost
7,Europa League,2019,490,0
6,Europa League,2018,519,0
5,Europa League,2017,205,0
4,Europa League,2016,205,0
3,Champions League,2019,203,0
2,Champions League,2018,216,0
1,Champions League,2017,219,0
0,Champions League,2016,217,0



Total UEFA matches in rounds of interest: (1278, 55) Breakdown:


...

,league_name,league_season,ngames,matches_lost
7,Europa League,2019,176,0
6,Europa League,2018,205,0
5,Europa League,2017,205,0
4,Europa League,2016,205,0
3,Champions League,2019,108,4
2,Champions League,2018,125,0
1,Champions League,2017,125,0
0,Champions League,2016,125,0



Total UEFA matches with at least some stats: (835, 55) Breakdown:
Total UEFA matches with stats in the corrected stats df: (835, 55)


,league_name,league_season,ngames,matches_lost
5,Europa League,2019,176,0
4,Europa League,2018,205,0
3,Champions League,2019,108,0
2,Champions League,2018,125,0
1,Champions League,2017,96,29
0,Champions League,2016,125,0


Showing how many null values are in our final dataset


,0
home_offsides,16
away_offsides,16
home_gksaves,3
away_gksaves,3


In [7]:
# Investigating how many UEFA matches dropped for not having data had teams of interest (df defined above)
# Operations to only keep matches of teams of interest
key_team_ids = u.get_ids_of_key_teams()
print("Total matches before filtering by what teams are playing:", uefa_dropped.shape)
display(pd.DataFrame(uefa_dropped.league_name.value_counts()))
# 1) Matches with at least one team of interest playing
uefa_dropped = uefa_dropped[(uefa_dropped['home_team_id'].isin(key_team_ids)) | (uefa_dropped['away_team_id'].isin(key_team_ids))].copy()
print("Total matches where at least one team is from one of the 5 major European Leagues:", uefa_dropped.shape)
display(pd.DataFrame(uefa_dropped.league_name.value_counts()))
# 2) Matches where both teams playing are of interest
uefa_dropped = uefa_dropped[(uefa_dropped['home_team_id'].isin(key_team_ids)) & (uefa_dropped['away_team_id'].isin(key_team_ids))].copy()
print("Total matches where BOTH teams are from one of the 5 major European Leagues:", uefa_dropped.shape)
display(pd.DataFrame(uefa_dropped.league_name.value_counts()))

Total matches before filtering by what teams are playing: (439, 55)


,league_name
Europa League,410
Champions League,29


Total matches where at least one team is from one of the 5 major European Leagues: (257, 55)


,league_name
Europa League,228
Champions League,29


Total matches where BOTH teams are from one of the 5 major European Leagues: (68, 55)


,league_name
Europa League,47
Champions League,21


## Matches that were not played

In [8]:
# How many games were not played due to postponement versus COVID-19
matches_unplayed = pd.read_csv(os.path.join(csv_json_data_dir, "raw/match_stats.csv"))
matches_unplayed = matches_unplayed[(matches_unplayed.league_type == "League") | (matches_unplayed.country == "World")]
print(matches_unplayed.shape)
matches_unplayed = matches_unplayed[matches_unplayed.league_season >= 2016]
print(matches_unplayed.shape)
matches_unplayed = matches_unplayed[~(matches_unplayed.fixture_status == "Match Finished")]
print(matches_unplayed.shape)
# Seperate unplayed matches by category. Group matches into those that were delayed and those from gameweeks where league play had already ceased.
matches_unplayed = matches_unplayed[matches_unplayed.fixture_status != "Match Cancelled"] # Two cancelled games unimportant
before_march = matches_unplayed.fixture_date.apply(lambda x: int(x.split("/")[1])<3)
during_march = matches_unplayed.fixture_date.apply(lambda x: int(x.split("/")[1])==3)
after_march = matches_unplayed.fixture_date.apply(lambda x: int(x.split("/")[1])>3)
before_twelfth = matches_unplayed.fixture_date.apply(lambda x: int(x.split("/")[0])<12)

matches_unplayed_postponed = matches_unplayed[before_march | (during_march & before_twelfth)]
matches_unplayed_covid = matches_unplayed[(after_march) | (during_march & (~before_twelfth))]

print(len(matches_unplayed_postponed) + len(matches_unplayed_covid) == len(matches_unplayed))
display("Postponed", pd.DataFrame(matches_unplayed_postponed.league_name.value_counts()))
print(matches_unplayed_postponed.league_name.value_counts().sum())
display("COVID-19", pd.DataFrame(matches_unplayed_covid.league_name.value_counts()))
print(matches_unplayed_covid.league_name.value_counts().sum())

(20528, 55)
(9582, 55)
(587, 55)
True


'Postponed'

,league_name
Serie A,26
Ligue 1,23
Bundesliga 1,16
Primera Division,15
Premier League,8


88


'COVID-19'

,league_name
Serie A,113
Primera Division,110
Ligue 1,101
Premier League,90
Bundesliga 1,81
Champions League,4


499


## Extra code to find important games with missing data

In [ ]:
match_stats_raw[(~match_stats_raw.has_match_stats) & (match_stats_raw.fixture_status == "Match Finished") & (match_stats_raw.league_type=="League") &(match_stats_raw.league_season>2015)]

# Data Description

In [11]:
# match_stats_clean = pd.read_csv(os.path.join(csv_json_data_dir, "clean/match_stats.csv"))

# Data below has included the domestic cup games of interest. To get the data from the line above just remove the condition in line 172 of etl.py (... | ((match_stats_csv.league_type == "Cup") & (match_stats_csv.country != "World")))
match_stats_clean2 = pd.read_csv(os.path.join(csv_json_data_dir, "clean/match_stats.csv"))

In [10]:
print("Data dimensions:", match_stats_clean.shape)
print("Key Competitions:", match_stats_clean[(match_stats_clean.country == "World") | (match_stats_clean.league_type == "League")].league_name.unique())
print("Number of Domestic Cup Competitions:", len(match_stats_clean[(match_stats_clean.country != "World") & (match_stats_clean.league_type=="Cup")].league_name.unique()))
print("Games per league:\n", match_stats_clean.groupby("league_name").count()[["fixture_id"]])
print("Games per season:\n", match_stats_clean.groupby("league_season").count()[["fixture_id"]])
print("Games per league season:\n", match_stats_clean.groupby(["league_season", "league_name"]).count()[["fixture_id"]])
print("Home games for most popular teams:\n", match_stats_clean.groupby(["home_team_name"]).count()[["fixture_id"]].reset_index().merge(match_stats_clean.groupby(["away_team_name"]).count()[["fixture_id"]].reset_index(), left_on="home_team_name", right_on="away_team_name", how="outer")[["home_team_name", "fixture_id_x", "fixture_id_y"]].sort_values(by="fixture_id_x", ascending = False).reset_index(drop=True).loc[:80,])
print("Most common scorelines:\n", match_stats_clean.groupby(["fixture_result_ft"]).count()[["fixture_id"]].sort_values(by="fixture_id", ascending=False))

Data dimensions: (7343, 54)
Key Competitions: ['Premier League' 'Bundesliga 1' 'Serie A' 'Primera Division' 'Ligue 1'
 'Champions League' 'Europa League']
Number of Domestic Cup Competitions: 0
Games per league:
                   fixture_id
league_name                 
Bundesliga 1            1144
Champions League         396
Europa League            207
Ligue 1                 1413
Premier League          1428
Primera Division        1371
Serie A                 1384
Games per season:
                fixture_id
league_season            
2016                 1932
2017                 1850
2018                 2056
2019                 1505
Games per league season:
                                 fixture_id
league_season league_name                 
2016          Bundesliga 1             306
              Champions League         107
              Ligue 1                  379
              Premier League           380
              Primera Division         380
              Serie A   

In [12]:
print("Data dimensions:", match_stats_clean2.shape)
print("Key Competitions:", match_stats_clean2[(match_stats_clean2.country == "World") | (match_stats_clean2.league_type == "League")].league_name.unique())
print("Number of Domestic Cup Competitions:", len(match_stats_clean2[(match_stats_clean2.country != "World") & (match_stats_clean2.league_type=="Cup")].league_name.unique()))
print("Games per league:\n", match_stats_clean2.groupby("league_name").count()[["fixture_id"]])
print("Games per season:\n", match_stats_clean2.groupby("league_season").count()[["fixture_id"]])
print("Games per league season:\n", match_stats_clean2.groupby(["league_season", "league_name"]).count()[["fixture_id"]])
print("Home games for most popular teams:\n", match_stats_clean2.groupby(["home_team_name"]).count()[["fixture_id"]].reset_index().merge(match_stats_clean2.groupby(["away_team_name"]).count()[["fixture_id"]].reset_index(), left_on="home_team_name", right_on="away_team_name", how="outer")[["home_team_name", "fixture_id_x", "fixture_id_y"]].sort_values(by="fixture_id_x", ascending = False).reset_index(drop=True).loc[:80,])
print("Most common scorelines:\n", match_stats_clean2.groupby(["fixture_result_ft"]).count()[["fixture_id"]].sort_values(by="fixture_id", ascending=False))

Data dimensions: (7606, 54)
Key Competitions: ['Premier League' 'Bundesliga 1' 'Serie A' 'Primera Division' 'Ligue 1'
 'Champions League' 'Europa League']
Number of Domestic Cup Competitions: 7
Games per league:
                    fixture_id
league_name                  
Bundesliga 1             1144
Champions League          396
Copa del Rey               51
Coppa Italia               54
Coupe de France            40
Coupe de la Ligue          21
DFB Pokal                  52
Europa League             207
FA Cup                     29
League Cup                 16
Ligue 1                  1413
Premier League           1428
Primera Division         1371
Serie A                  1384
Games per season:
                fixture_id
league_season            
2016                 1987
2017                 1868
2018                 2177
2019                 1574
Games per league season:
                                  fixture_id
league_season league_name                  
2016          Bund

In [60]:
match_stats_clean.head()

,fixture_id,country,league_name,league_id,league_type,league_season,fixture_date,fixture_round,fixture_status,fixture_elapsed,fixture_venue,fixture_referee,fixture_result_ht,fixture_result_ft,fixture_result_et,fixture_result_pen,home_team_name,home_team_id,away_team_name,away_team_id,home_goals,away_goals,home_shots_ont,away_shots_ont,home_shots_offt,away_shots_offt,home_shots_bl,away_shots_bl,home_shots_tot,away_shots_tot,home_shots_inb,away_shots_inb,home_shots_outb,away_shots_outb,home_passes_acc,away_passes_acc,home_passes_tot,away_passes_tot,home_passes_pct,away_passes_pct,home_possession,away_possession,home_corners,away_corners,home_offsides,away_offsides,home_fouls,away_fouls,home_yc,away_yc,home_rc,away_rc,home_gksaves,away_gksaves
0,65,England,Premier League,2,League,2018,10/08/2018,Regular Season - 1,Match Finished,90,"Old Trafford, Manchester","Andre Marriner, England",1-0,2-1,NaN,NaN,Manchester United,33,Leicester,46,2.0,1.0,6.0,4.0,1.0,3.0,1.0,6.0,8.0,13.0,5.0,5.0,3.0,8.0,399.0,450.0,485.0,543.0,0.82,0.83,0.46,0.54,2.0,5.0,4.0,2.0,11.0,8.0,2.0,1.0,0.0,0.0,3.0,4.0
1,66,England,Premier League,2,League,2018,11/08/2018,Regular Season - 1,Match Finished,90,"St James' Park, Newcastle upon Tyne","Martin Atkinson, England",1-2,1-2,NaN,NaN,Newcastle,34,Tottenham,47,1.0,2.0,2.0,5.0,8.0,7.0,5.0,3.0,15.0,15.0,9.0,12.0,6.0,3.0,268.0,467.0,387.0,573.0,0.69,0.82,0.40,0.60,3.0,5.0,1.0,0.0,11.0,12.0,2.0,2.0,0.0,0.0,3.0,1.0
2,67,England,Premier League,2,League,2018,11/08/2018,Regular Season - 1,Match Finished,90,"Vitality Stadium, Bournemouth","Kevin Friend, England",1-0,2-0,NaN,NaN,Bournemouth,35,Cardiff,43,2.0,0.0,4.0,1.0,6.0,5.0,2.0,4.0,12.0,10.0,10.0,8.0,2.0,2.0,397.0,175.0,502.0,287.0,0.79,0.61,0.62,0.38,7.0,4.0,0.0,2.0,11.0,9.0,1.0,1.0,0.0,0.0,1.0,2.0
3,68,England,Premier League,2,League,2018,11/08/2018,Regular Season - 1,Match Finished,90,"Craven Cottage, London","Mike Dean, England",0-1,0-2,NaN,NaN,Fulham,36,Crystal Palace,52,0.0,2.0,6.0,10.0,4.0,0.0,5.0,2.0,15.0,12.0,6.0,8.0,9.0,4.0,591.0,258.0,672.0,347.0,0.88,0.74,0.66,0.34,5.0,5.0,2.0,3.0,9.0,11.0,1.0,2.0,0.0,0.0,8.0,6.0
4,69,England,Premier League,2,League,2018,11/08/2018,Regular Season - 1,Match Finished,90,"John Smith's Stadium, Huddersfield","Chris Kavanagh, England",0-2,0-3,NaN,NaN,Huddersfield,37,Chelsea,49,0.0,3.0,1.0,4.0,4.0,3.0,1.0,6.0,6.0,13.0,4.0,10.0,2.0,3.0,281.0,582.0,372.0,658.0,0.76,0.88,0.37,0.63,2.0,5.0,2.0,1.0,9.0,8.0,2.0,1.0,0.0,0.0,1.0,1.0


In [50]:
# # key_team_ids = u.get_ids_of_key_teams()
# [x for x in match_stats_raw[(match_stats_raw.league_type=="Cup") & (match_stats_raw.country!="World") & (match_stats_raw.fixture_round == "32nd Finals")].home_team_id.unique() if str(x) in key_team_ids]
# all_teams = pd.read_csv("../data/csv_of_json_data/all_teams.csv")
# all_teams[all_teams.team_id == 80]